In [19]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Script Needs

In [20]:
from time import sleep

import twitter, re, datetime, pandas as pd
import sys
import re
import time
import tweepy

import os

import pickle
from sklearn.linear_model import LogisticRegression

In [21]:
cvec = pickle.load(open('vectorizer.sav', 'rb')) 
lr = pickle.load(open('lin_regressor.sav', 'rb')) 

In [22]:
# One time
with open('./twitterapi.txt') as f:
    ck, cs, atk, ats = f.read().split(',')

In [23]:
# your keys go here:
twitter_keys = {
    'consumer_key':        ck,
    'consumer_secret':     cs,
    'access_token_key':    atk,
    'access_token_secret': ats
}

In [24]:
auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [25]:
def hour_converter(_time_input):
    _time_zone_adjustment = 6
    if _time_input < _time_zone_adjustment:
        _hour_statement = str(_time_input - _time_zone_adjustment + 24) + ":00 PM"
    elif _time_input == _time_zone_adjustment:
        _hour_statement = "12:00 AM"
    elif _time_input < 19: 
        _hour_statement = str(_time_input - _time_zone_adjustment) + ":00 AM"
    else:
        _hour_statement = str(_time_input - _time_zone_adjustment) + ":00 PM"
    return _hour_statement

In [26]:
# update_interval_hours = 6
# for i in range(24):
#     sleep(update_interval_hours*60*60)

In [27]:
tomorrow = datetime.datetime.now() + datetime.timedelta(1)
today = datetime.datetime.now()
yesterday = datetime.datetime.now() - datetime.timedelta(1)
end_date = '{}-{}-{}'.format(tomorrow.year, tomorrow.month, tomorrow.day)   
start_date = '{}-{}-{}'.format(yesterday.year, yesterday.month, yesterday.day)

In [28]:
# Otherwise, won't update public doc for 1 hour
# !gsutil -h "Cache-Control:no-cache, max-age=0" cp -a public-read myfile.json gs://mybucket

In [29]:
df_small = pd.DataFrame()
places = ["Austin, TX"]
tweetsPerQry = 100
counter = 0

for place_q in places: 

    place = api.geo_search(query=place_q, granularity="city")
    place_id = place[0].id

    max_id = -1
    for _ in range(10):
        try:
            if (max_id <= 0):
                new_tweets = api.search(q='place:%s' % place_id, count=tweetsPerQry, since=start_date, until=end_date)
            else:
                new_tweets = api.search(q='place:%s' % place_id, count=tweetsPerQry, max_id=str(max_id - 1), since=start_date, until=end_date)

            df_text = pd.DataFrame([new_tweets[i]._json['text'] for i in range(len(new_tweets))], columns=['tweet_text'])
            df_text['tweet_time']=[new_tweets[i]._json['created_at'] for i in range(len(new_tweets))]
            df_text['tweet_place'] = place_q

            df_small = pd.concat([df_small, df_text])
            if not new_tweets:
                counter += 1
                df_small.to_csv('./data/collected_tweets_{}.csv'.format(counter)) # Should provide log of last good pull at least
                print("No more tweets found")
                break

            max_id = new_tweets[-1].id
        except tweepy.TweepError as e:
            print('    all_done')
            break
        sleep(5)

In [32]:
# Try to use only the most recent tweets
# If there are not enough, use a wider range
df_small["tweet_time"] = pd.to_datetime(df_small.tweet_time)
if df_small[df_small.tweet_time > (datetime.datetime.now() - datetime.timedelta(hours=2))].shape[0] > 50:
    df_small = df_small[df_small.tweet_time > (datetime.datetime.now() - datetime.timedelta(hours=2))]
elif df_small == df_small[df_small.tweet_time > (datetime.datetime.now() - datetime.timedelta(hours=4))].shape[0]>50:
    df_small = df_small[df_small.tweet_time > (datetime.datetime.now() - datetime.timedelta(hours=4))]
else:
    df_small = df_small

In [33]:
model_columns = cvec.get_feature_names()
df_small = pd.concat([df_small.reset_index().drop('index', axis=1), pd.DataFrame(cvec.transform(df_small.tweet_text.str.lower()).todense(), columns=cvec.get_feature_names())], axis=1)
_X = df_small.loc[:, [col for col in df_small.columns if col in model_columns]]

_X.fillna(0, inplace=True)

df_small['predicted'] = lr.predict(_X)
df_small['probas'] = [element[1] for element in lr.predict_proba(_X)]

In [34]:
excludes = ['tweet_text', 'tweet_time', 'tweet_place', 'is_rain', 'predicted']
_rain_probability = df_small.loc[:, excludes + ['predicted', 'probas']].groupby('tweet_place').mean()['probas'].values[0]
_rain_probability = round(_rain_probability*100, 1)

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [35]:
_rain_probability

59.0

In [36]:
# Write qualifying statement separately for styling
if _rain_probability < 20:
    rain_qualifier = "(so probably not)"
elif _rain_probability < 25:
    rain_qualifier = "(so maybe?)"
elif _rain_probability < 80:
    rain_qualifier = "(so probably!)"
else:
    rain_qualifier = "Its definitely raining!"

In [37]:
html_statement = '<!DOCTYPE html><html><head><style type="text/css">.karen_message{}</style><style type="text/css">.qualifier_message{} </style></head><body><center><p class="karen_message">This is Karen_Bot with the weather! <br><br>I am here in Austin and there is a <br><strong>{}%</strong> chance that it is already raining! <br><br><i class="qualifier_message">{}</i> </p></center> </body></html>'.format("{font-size: 16px};","{font-size: 12px};", _rain_probability, rain_qualifier)

In [38]:
html_statement

'<!DOCTYPE html><html><head><style type="text/css">.karen_message{font-size: 16px};</style><style type="text/css">.qualifier_message{font-size: 12px}; </style></head><body><center><p class="karen_message">This is Karen_Bot with the weather! <br><br>I am here in Austin and there is a <br><strong>59.0%</strong> chance that it is already raining! <br><br><i class="qualifier_message">(so probably!)</i> </p></center> </body></html>'

In [39]:
# Write text doc

In [28]:
f= open("isitraining.html","w+")
f.write(html_statement)
f.close() 

In [22]:
'{} {}, {} at {}'.format(today.strftime("%B"), _day, _year, hour_converter(_hour))

'January 23, 2019 at 18:00 PM'

In [23]:
f= open("update_date.txt","w+")
f.write('{} {}, {} at {}'.format(datetime.datetime.now().strftime("%B"), _day, _year, hour_converter(_hour)))
f.close() 

In [21]:
# #set file private
# os.system("gsutil acl set private gs://is-it-raining/isitraining.txt")

# # Delete existing
# os.system("gsutil rm gs://is-it-raining/isitraining.txt")

# copy in new
os.system("gsutil cp -r ~/isitraining.html gs://is-it-raining/")
os.system("gsutil cp -r ~/update_date.txt gs://is-it-raining/")

# set access public
os.system("gsutil acl ch -u AllUsers:R gs://is-it-raining/isitraining.html")
os.system("gsutil acl ch -u AllUsers:R gs://is-it-raining/update_date.txt")

0

In [18]:
# Update the log to record date, time, and prediction
update_log = pd.read_csv('./update_log.csv')
update_log = pd.concat([update_log, pd.DataFrame({'Date': datetime.datetime.now(), 'rain_statement': _rain_probability}, index=[0])])
update_log.to_csv('./update_log.csv', index=False)